In [1]:
!pip install fastapi uvicorn transformers beautifulsoup4 pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.6 MB/s eta 0:00:00


In [5]:
%%writefile main.py
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import requests
from bs4 import BeautifulSoup

app = FastAPI()

# Load the GPT-Neo model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B")

class BlogInput(BaseModel):
    urls_and_indices: list
    no_words: int

# Function to scrape specific paragraphs from a webpage
def scrape_specific_paragraphs(url, indices):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all('p')

    selected_paragraphs = []
    for index in indices:
        if index < len(paragraphs):
            selected_paragraphs.append(paragraphs[index].get_text())
        else:
            selected_paragraphs.append(f"Paragraph index {index} out of range.")

    return selected_paragraphs

@app.post("/generate_blog")
async def generate_blog(input: BlogInput):
    results = []
    for url, indices in input.urls_and_indices:
        specific_paragraphs = scrape_specific_paragraphs(url, indices)
        # Prepare the blog generation template
        template = f"""
        You are a blog writer. Generate a blog within {input.no_words} words based on the following content: {' '.join(specific_paragraphs)}
        Structure:
        1. **Introduction**: Briefly introduce the topic.
        2. **Main Points**: Discuss the significance, various perspectives, and practical applications.
        3. **Conclusion**: Summarize the main ideas.
        """
        inputs = tokenizer(template, return_tensors="pt", max_length=700, truncation=True)
        output = model.generate(**inputs, max_length=700, num_return_sequences=1)
        blog = tokenizer.decode(output[0], skip_special_tokens=True)
        results.append(blog)
    return {"blogs": results}

Overwriting main.py
INFO:     Shutting down


In [6]:
import threading

def run_fastapi():
    !uvicorn main:app --host 0.0.0.0 --port 8000 --reload

# Run the FastAPI server in a separate thread
thread = threading.Thread(target=run_fastapi)
thread.start()

INFO:     Started server process [1864]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Will watch for changes in these directories: ['/content']
ERROR:    [Errno 98] Address already in use


In [7]:
from pyngrok import ngrok

# Set the FastAPI port
port = 8000
ngrok.set_auth_token("2iZTB0eNxBUBRt0WYIlhQeAp1dv_6xHYVAjXCDqn14U9Bek9y")  # Replace with your ngrok auth token
public_url = ngrok.connect(port)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://localhost:{port}\"")

 * ngrok tunnel "NgrokTunnel: "https://9205-34-123-190-61.ngrok-free.app" -> "http://localhost:8000"" -> "http://localhost:8000"


In [ ]:
# Create main.py i.e for FASTAPI endpoints which consist of uploading the model, scrapping the text, create endpoint for blog generation which consist of prompt template.
# Then used ngork to expose the backend part of the fastapi endpoints and generating URL.
# Then used streamlit locally to create app.py (http://192.168.204.13:8501) based on the Ngork URL.